In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd
import matplotlib.path as mpath

%matplotlib inline

In [ ]:
ds = xr.open_dataset("/glade/derecho/scratch/ksha/CREDIT/GATHER/wxformer_6h/2020-04-12T00Z.nc")

In [ ]:
for t in range(4):
    fig = plt.figure(figsize=(20, 12), dpi=300)
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.EckertIII())
    ax.set_global()
    ax.coastlines("110m", alpha=0.5)
    heights = ax.contour(
        ds["lon"],
        ds["lat"],
        ds["Z500"][t] / 9.81 / 10,
        np.arange(450, 600, 6),
        transform=ccrs.PlateCarree(),
        cmap="Greys",
        linewidths=0.5,
    )
    wspd = np.sqrt(ds["U500"][t] ** 2 + ds["V500"][t] ** 2) * 1.94384
    ax.contourf(
        ds["lon"],
        ds["lat"],
        wspd,
        np.arange(30, 150, 10),
        transform=ccrs.PlateCarree(),
        cmap="YlOrRd",
        extend="max",
    )
    ax.contourf(
        ds["lon"],
        ds["lat"],
        ds["Q500"][t] * 1000,
        np.arange(1, 6, 1),
        transform=ccrs.PlateCarree(),
        cmap="viridis",
        extend="max",
    )
    ax.contour(
        ds["lon"],
        ds["lat"],
        ds["T500"][t] - 273,
        np.arange(-50, 5, 5),
        transform=ccrs.PlateCarree(),
        cmap="RdBu_r",
        extend="max",
    )
    time_str = pd.Timestamp(ds.time[t].values).strftime("%Y-%m-%d %H:%M")
    time_str_f = pd.Timestamp(ds.time[t].values).strftime("%Y%m%d_%H%M")

    ax.set_title(f"500 mb WxFormer Valid {time_str}")
    plt.savefig(f"wxformer_500_{time_str_f}.png", dpi=300, bbox_inches="tight")

In [ ]:
def polarCentral_set_latlim(lat_lims, ax):
    ax.set_extent([-180, 180, lat_lims[0], lat_lims[1]], ccrs.PlateCarree())
    # Compute a circle in axes coordinates, which we can use as a boundary
    # for the map. We can pan/zoom as much as we like - the boundary will be
    # permanently circular.
    theta = np.linspace(0, 2 * np.pi, 100)
    center, radius = [0.5, 0.5], 0.5
    verts = np.vstack([np.sin(theta), np.cos(theta)]).T
    circle = mpath.Path(verts * radius + center)

    ax.set_boundary(circle, transform=ax.transAxes)


lat_lims = [10, 90]
lat_i = 284
for t in range(4):
    fig = plt.figure(figsize=(20, 12), dpi=300)
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.NorthPolarStereo(central_longitude=35.0))
    # ax.coastlines('110m')
    ax.add_feature(cfeature.OCEAN, color="lightblue")
    ax.add_feature(cfeature.LAND, color="0.5")
    polarCentral_set_latlim(lat_lims, ax)
    heights = ax.contour(
        ds["lon"],
        ds["lat"][:lat_i],
        ds["Z500"][t, :lat_i] / 9.81 / 10,
        np.arange(450, 600, 6),
        transform=ccrs.PlateCarree(),
        colors="k",
        linewidths=0.75,
    )
    ax.clabel(heights)
    wspd = np.sqrt(ds["U500"][t, :lat_i] ** 2 + ds["V500"][t, :lat_i] ** 2) * 1.94384
    ax.contourf(
        ds["lon"],
        ds["lat"][:lat_i],
        wspd,
        np.arange(30, 150, 10),
        transform=ccrs.PlateCarree(),
        cmap="YlOrRd",
        extend="max",
    )
    ax.contourf(
        ds["lon"],
        ds["lat"][:lat_i],
        ds["Q500"][t, :lat_i] * 1000,
        np.arange(1, 6, 1),
        transform=ccrs.PlateCarree(),
        cmap="viridis",
        extend="max",
    )
    temps = ax.contour(
        ds["lon"],
        ds["lat"][:lat_i],
        ds["T500"][t, :lat_i] - 273,
        np.arange(-50, 5, 5),
        transform=ccrs.PlateCarree(),
        cmap="RdBu_r",
        extend="max",
    )
    ax.clabel(temps)
    time_str = pd.Timestamp(ds.time[t].values).strftime("%Y-%m-%d %H:%M")
    time_str_f = pd.Timestamp(ds.time[t].values).strftime("%Y%m%d_%H%M")

    ax.set_title(f"500 mb WxFormer Valid {time_str}")
    plt.savefig(f"wxformer_np_500_{time_str_f}.png", dpi=300, bbox_inches="tight")

In [ ]:
np.where(ds["lat"] >= 10)

In [ ]:
ds["lat"]

In [ ]:
ds

In [ ]:
ds["SP"].min()

In [ ]:
for t in range(4):
    fig = plt.figure(figsize=(20, 12), dpi=300)
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.EckertIII())
    ax.set_global()
    ax.coastlines("110m", alpha=0.5)
    heights = ax.contour(
        ds["lon"],
        ds["lat"],
        ds["SP"][t] / 100,
        np.arange(500, 1100, 20),
        transform=ccrs.PlateCarree(),
        cmap="Greys",
        linewidths=0.5,
    )
    ax.contour(
        ds["lon"],
        ds["lat"],
        ds["t2m"][t] - 273,
        np.arange(-40, 40, 5),
        transform=ccrs.PlateCarree(),
        cmap="RdBu_r",
        extend="max",
    )
    time_str = pd.Timestamp(ds.time[t].values).strftime("%Y-%m-%d %H:%M")
    time_str_f = pd.Timestamp(ds.time[t].values).strftime("%Y%m%d_%H%M")

    ax.set_title(f"Surface WxFormer Valid {time_str}")
    plt.savefig(f"wxformer_surface_{time_str_f}.png", dpi=300, bbox_inches="tight")

In [1]:
era5_ds = xr.open_zarr(
    "/glade/campaign/cisl/aiml/wchapman/MLWPS/STAGING/SixHourly_y_TOTAL_2020-01-01_2020-12-31_staged.zarr"
)

NameError: name 'xr' is not defined

In [ ]:
t_idx = np.where(era5_ds["time"] == ds["time"][0])[0][0]
for t in range(t_idx, t_idx + 4):
    fig = plt.figure(figsize=(20, 12), dpi=300)
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.EckertIII())
    ax.set_global()
    ax.coastlines("110m", alpha=0.5)
    heights = ax.contour(
        era5_ds["longitude"],
        era5_ds["latitude"],
        era5_ds["Z500"][t] / 9.81 / 10,
        np.arange(450, 600, 6),
        transform=ccrs.PlateCarree(),
        cmap="Greys",
        linewidths=0.5,
    )
    wspd = np.sqrt(era5_ds["U500"][t] ** 2 + era5_ds["V500"][t] ** 2) * 1.94384
    ax.contourf(
        era5_ds["longitude"],
        era5_ds["latitude"],
        wspd,
        np.arange(30, 150, 10),
        transform=ccrs.PlateCarree(),
        cmap="YlOrRd",
        extend="max",
    )
    ax.contourf(
        era5_ds["longitude"],
        era5_ds["latitude"],
        era5_ds["Q500"][t] * 1000,
        np.arange(1, 6, 1),
        transform=ccrs.PlateCarree(),
        cmap="viridis",
        extend="max",
    )
    ax.contour(
        era5_ds["longitude"],
        era5_ds["latitude"],
        era5_ds["T500"][t] - 273,
        np.arange(-50, 5, 5),
        transform=ccrs.PlateCarree(),
        cmap="RdBu_r",
        extend="max",
    )
    time_str = pd.Timestamp(era5_ds.time[t].values).strftime("%Y-%m-%d %H:%M")
    time_str_f = pd.Timestamp(era5_ds.time[t].values).strftime("%Y%m%d_%H%M")

    ax.set_title(f"500 mb ERA5 Valid {time_str}")
    plt.savefig(f"era5_500_{time_str_f}.png", dpi=300, bbox_inches="tight")

In [ ]:
def polarCentral_set_latlim(lat_lims, ax):
    ax.set_extent([-180, 180, lat_lims[0], lat_lims[1]], ccrs.PlateCarree())
    # Compute a circle in axes coordinates, which we can use as a boundary
    # for the map. We can pan/zoom as much as we like - the boundary will be
    # permanently circular.
    theta = np.linspace(0, 2 * np.pi, 100)
    center, radius = [0.5, 0.5], 0.5
    verts = np.vstack([np.sin(theta), np.cos(theta)]).T
    circle = mpath.Path(verts * radius + center)

    ax.set_boundary(circle, transform=ax.transAxes)


lat_lims = [10, 90]
lat_i = 284
t_idx = np.where(era5_ds["time"] == ds["time"][0])[0][0]
for t in range(t_idx, t_idx + 4):
    fig = plt.figure(figsize=(20, 12), dpi=300)
    ax = fig.add_subplot(1, 1, 1, projection=ccrs.NorthPolarStereo(central_longitude=35.0))
    # ax.coastlines('110m')
    ax.add_feature(cfeature.OCEAN, color="lightblue")
    ax.add_feature(cfeature.LAND, color="0.5")
    polarCentral_set_latlim(lat_lims, ax)
    heights = ax.contour(
        era5_ds["longitude"],
        era5_ds["latitude"][:lat_i],
        era5_ds["Z500"][t, :lat_i] / 9.81 / 10,
        np.arange(450, 600, 6),
        transform=ccrs.PlateCarree(),
        colors="k",
        linewidths=0.75,
    )
    ax.clabel(heights)
    wspd = np.sqrt(era5_ds["U500"][t, :lat_i] ** 2 + era5_ds["V500"][t, :lat_i] ** 2) * 1.94384
    ax.contourf(
        era5_ds["longitude"],
        era5_ds["latitude"][:lat_i],
        wspd,
        np.arange(30, 150, 10),
        transform=ccrs.PlateCarree(),
        cmap="YlOrRd",
        extend="max",
    )
    ax.contourf(
        era5_ds["longitude"],
        era5_ds["latitude"][:lat_i],
        era5_ds["Q500"][t, :lat_i] * 1000,
        np.arange(1, 6, 1),
        transform=ccrs.PlateCarree(),
        cmap="viridis",
        extend="max",
    )
    temps = ax.contour(
        era5_ds["longitude"],
        era5_ds["latitude"][:lat_i],
        era5_ds["T500"][t, :lat_i] - 273,
        np.arange(-50, 5, 5),
        transform=ccrs.PlateCarree(),
        cmap="RdBu_r",
        extend="max",
    )
    ax.clabel(temps)
    time_str = pd.Timestamp(era5_ds.time[t].values).strftime("%Y-%m-%d %H:%M")
    time_str_f = pd.Timestamp(era5_ds.time[t].values).strftime("%Y%m%d_%H%M")

    ax.set_title(f"500 mb ERA5 Valid {time_str}")
    plt.savefig(f"era5_np_500_{time_str_f}.png", dpi=300, bbox_inches="tight")

In [ ]:
era5_climo = xr.open_dataset("/glade/derecho/scratch/wchapman/ERA5_Climo.nc")

In [ ]:
era5_climo

In [ ]:
ax.add_feature(cfeature.LAND)